In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
tf.random.set_seed(1)

rnn_layer = tf.keras.layers.SimpleRNN(
    units=2, use_bias=True,
    return_sequences=True
)
rnn_layer.build(input_shape=(None, None, 5))

w_xh, w_oo, b_h = rnn_layer.weights

print('W_xh shape:', w_xh.shape)
print('W_oo shape:', w_oo.shape)
print('b_h shape:', b_h.shape)

2023-08-08 15:36:38.679012: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


W_xh shape: (5, 2)
W_oo shape: (2, 2)
b_h shape: (2,)


2023-08-08 15:36:42.080366: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
x_seq = tf.convert_to_tensor(
    [[1.] * 5, [2.] * 5, [3.] * 5],
    dtype=tf.float32
)

output = rnn_layer(tf.reshape(x_seq, shape=(1, 3, 5)))

out_man = []
for t in range(len(x_seq)):
    xt = tf.reshape(x_seq[t], (1, 5))
    print(f'Time step {t} =>')
    print('   Input           :', xt.numpy())
    
    ht = tf.matmul(xt, w_xh) + b_h
    print('   Hidden          :', ht.numpy())
    
    if t > 0:
        pre_o = out_man[t - 1]
    else:
        pre_o = tf.zeros(shape=ht.shape)
    
    ot = ht + tf.matmul(pre_o, w_oo)
    ot = tf.math.tanh(ot)
    out_man.append(ot)
    print('   Output (manual) :', ot.numpy())
    print('   SimpleRNN {} output:'.format(t), output[0][t].numpy())
    print()
        

Time step 0 =>
   Input           : [[1. 1. 1. 1. 1.]]
   Hidden          : [[ 0.12166828 -1.1690187 ]]
   Output (manual) : [[ 0.12107145 -0.8239574 ]]
   SimpleRNN 0 output: [ 0.12107145 -0.8239574 ]

Time step 1 =>
   Input           : [[2. 2. 2. 2. 2.]]
   Hidden          : [[ 0.24333656 -2.3380375 ]]
   Output (manual) : [[-0.07423016 -0.91674113]]
   SimpleRNN 1 output: [-0.07423016 -0.91674113]

Time step 2 =>
   Input           : [[3. 3. 3. 3. 3.]]
   Hidden          : [[ 0.36500466 -3.5070555 ]]
   Output (manual) : [[ 0.21097866 -0.98902225]]
   SimpleRNN 2 output: [ 0.21097866 -0.98902225]


# Implementing RNNs for sequence modeling in TensorFlow

## Project one: predicting the sentiment of IMDb movie reviews

### Preparing the movie review data

In [3]:
df = pd.read_csv('../datasets/movie_data.csv', encoding='utf-8')
df.tail()

,review,sentiment
49995,"OK, lets start with the best. the building. al...",0
49996,The British 'heritage film' industry is out of...,0
49997,I don't even know where to begin on this one. ...,0
49998,Richard Tyler is a little boy who is scared of...,0
49999,I waited long to watch this movie. Also becaus...,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [5]:
# Step 1: Create a dataset

ds_raw = tf.data.Dataset.from_tensor_slices((df.review, df.sentiment))

## inspection
for ex in ds_raw.take(3):
    tf.print(ex[0].numpy()[:50], ex[1])

b'In 1974, the teenager Martha Moxley (Maggie Grace)' 1
b'OK... so... I really like Kris Kristofferson and h' 0
b'***SPOILER*** Do not read this, if you think about' 0


In [6]:
# Train/validation/test splits

tf.random.set_seed(1)

ds_raw = ds_raw.shuffle(50000, reshuffle_each_iteration=False)
ds_raw_test = ds_raw.take(25000)
ds_raw_train_valid = ds_raw.take(25000)
ds_raw_train = ds_raw_train_valid.take(20000)
ds_raw_valid = ds_raw_train_valid.skip(20000)

In [28]:
# Step 2: find unique tokens (words)

from collections import Counter

tokenizer = tf.keras.preprocessing.text.Tokenizer()

tokenizer.fit_on_texts(next(iter(ds_raw_train))[0])

TypeError: Cannot iterate over a scalar tensor.